In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import json
import urllib.request as urllib2
import random
from random import choice
import random
import time
from datetime import datetime
from IPython.display import clear_output

def urlquery(url):
        # function cycles randomly through different user agents and time intervals to simulate more natural queries
        try:
            sleeptime = float(random.randint(1,6))/5
            #time.sleep(sleeptime)

            agents = ['Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1309.0 Safari/537.17',
            'Mozilla/5.0 (compatible; MSIE 10.6; Windows NT 6.1; Trident/5.0; InfoPath.2; SLCC1; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729; .NET CLR 2.0.50727) 3gpp-gba UNTRUSTED/1.0',
            'Opera/12.80 (Windows NT 5.1; U; en) Presto/2.10.289 Version/12.02',
            'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)',
            'Mozilla/3.0',
            'Mozilla/5.0 (iPhone; U; CPU like Mac OS X; en) AppleWebKit/420+ (KHTML, like Gecko) Version/3.0 Mobile/1A543a Safari/419.3',
            'Mozilla/5.0 (Linux; U; Android 0.5; en-us) AppleWebKit/522+ (KHTML, like Gecko) Safari/419.3',
            'Opera/9.00 (Windows NT 5.1; U; en)']

            agent = choice(agents)
            opener = urllib2.build_opener()
            opener.addheaders = [('User-agent', agent)]

            html = opener.open(url).read()
            #time.sleep(sleeptime)

            return html

        except Exception as e:
            print('Something went wrong with Crawling:\n%s' % e)
            
def create_review_deets_dict(user_id="",whisky_id="",points_overall="",nose_text="",taste_text="",finish_text="",nose_rating="",taste_rating="",finish_rating=""):
    review_dict = {
        'user_id':user_id,
        'whisky_id':whisky_id,
        'points_overall': points_overall,
        'nose_text': nose_text,
        'taste_text': taste_text,
        'finish_text': finish_text,
        'nose_rating': nose_rating,
        'taste_rating': taste_rating,
        'finish_rating': finish_rating
    }
    return review_dict

In [2]:
whisky_id=1
url='https://www.whiskybase.com/whiskies/whisky/'+str(whisky_id)
html=urlquery(url)
soup = BeautifulSoup(html, "lxml")
url

'https://www.whiskybase.com/whiskies/whisky/1'

## get all whisky deets

In [3]:
whiskies=[]
reviews_extract=[]
start=datetime.now()
#max_page=174482
max_page=10
csv_chunk=10
starting_page=1

for whisky_id in range(starting_page,max_page+1):
    try:
        url='https://www.whiskybase.com/whiskies/whisky/'+str(whisky_id)
        html=urlquery(url)
        soup = BeautifulSoup(html, "lxml")


        cleaned_id_text = []
        for i in soup.find_all('dt'):
            cleaned_id_text.append(i.text.replace('\n',''))
        cleaned_id__attrb_text = []
        for i in soup.find_all('dd'):
            cleaned_id__attrb_text.append(i.text.replace('\n',''))

        if len(cleaned_id_text)!=len(cleaned_id__attrb_text):
            cleaned_id__attrb_text.pop()
        else:
            pass

        avg_price=soup.find_all('div', {'class':'block-price'})[0]
        cleaned_id_text.append(avg_price.text.split('\n')[1])
        cleaned_id__attrb_text.append(avg_price.text.split('\n')[2])
        
        cleaned_id_text.append('id')
        cleaned_id__attrb_text.append(whisky_id)
        
        tag_values=[]
        tastingtags=soup.find_all('ul',{'class':'tastingtags'})[0]
        for item in tastingtags.find_all(attrs={"data-id": True}):
        #    if item['data-num'] != "":
            tag_values.append(item['data-id']+':'+item['data-num'])
        cleaned_id_text.append('tastingtags')
        cleaned_id__attrb_text.append(str(sorted(tag_values)).replace('[','').replace(']',''))
        
        whisky_deets=dict(zip(cleaned_id_text, cleaned_id__attrb_text))
        whiskies.append(whisky_deets)
        
        ## reviews ##
        reviews=soup.find_all('ol',{'class':'wb--notes'})
        for ol in reviews:
            for review in ol.findAll('article'):
                title=review.find_all('h5',{'class':"wb--note-title"})[0]['aria-label']
                user_id=review.find_all('h5',{'class':"wb--note-title"})[0].find_all('a')[0].contents[0]
                try:
                    points_overall=int(review.find_all('h5',{'class':"wb--note-title"})[0].find_all('span')[0].contents[0])
                except:
                    points_overall=np.NaN
                try:
                    nose_text=review.find_all('div',{'data-translation-field':"nose_text"})[0].text.replace('\n\t\t\t','').replace('\n\t\t','')
                    taste_text=review.find_all('div',{'data-translation-field':"taste_text"})[0].text.replace('\n\t\t\t','').replace('\n\t\t','')
                    finish_text=review.find_all('div',{'data-translation-field':"finish_text"})[0].text.replace('\n\t\t\t','').replace('\n\t\t','')
                except:
                    nose_text=np.NaN
                    taste_text=np.NaN
                    finish_text=np.NaN

                note_ratings=review.find_all('div',{'class':"wb--note-refinement-rating"})
                rating_list=[]
                for rating in note_ratings:
                    rating_type=rating.find('h6').text
                    try:
                        rating_value=rating.find('meter')['value']
                    except:
                        rating_value=np.NaN
                    rating_list.append(list([rating_type,rating_value]))

                ratings_dict={}
                for i in range(len(rating_list)):
                    if rating_list[i][0] in ratings_dict.keys():# if key is present in the list, just append the value
                        ratings_dict[rating_list[i][0]]=""
                    else:
                        ratings_dict[rating_list[i][0]]=rating_list[i][1] # now append the value for that key

                try:
                    nose_rating = ratings_dict['Nose']
                    taste_rating= ratings_dict['Taste']
                    finish_rating= ratings_dict['Finish']
                except:
                    nose_rating=np.NaN
                    taste_rating=np.NaN
                    finish_rating=np.NaN


                review_deets=create_review_deets_dict(user_id=user_id,
                     whisky_id=whisky_id,
                     points_overall=points_overall,
                     nose_text=nose_text,
                     taste_text=taste_text,
                     finish_text=finish_text,
                     nose_rating= nose_rating,
                     taste_rating= taste_rating,
                     finish_rating= finish_rating
                    )
                reviews_extract.append(review_deets)

        if (whisky_id%csv_chunk == 0):
            clear_output()
            print(whisky_id)
            current=datetime.now()
            progress=(whisky_id/max_page)
            duration=(current-start).seconds
            print(str(round(progress*100,2))+'%')
            print(str(int((duration)/60))+' minutes elapsed')
            print(str(int((duration/progress-duration)/60))+' minutes remaining')
            print(str(int((duration/progress-duration)/60/60))+' hours remaining')
            date=str(datetime.now().date())
            whiskies_df=pd.DataFrame(whiskies)
            whiskies_df.to_csv(f'whiskies:{date}.csv', index=False)
            review_df=pd.DataFrame(reviews_extract)
            review_df.to_csv(f'reviews:{date}.csv', index=False)
        else:
            pass        
    except:
        pass

review_df=pd.DataFrame(reviews_extract)
review_df.to_csv(f'reviews:{date}.csv', index=False)
whiskies_df=pd.DataFrame(whiskies)
whiskies_df.to_csv(f'whiskies:{date}.csv', index=False)

10
100.0%
0 minutes elapsed
0 minutes remaining
0 hours remaining
